In [17]:
import pandas as pd
import numpy as np
import mysql.connector
import re
import ast

In [18]:
top_250 = pd.read_csv('top_250.csv')

In [19]:
top_250['parental_guide'].replace(['blank' , 'Not Rated' , 'null' , 'NaN' , 'None'] , 'Unrated' , inplace=True)
top_250['parental_guide'].fillna('Unrated' , inplace=True)

In [20]:
top_250['gross_us_canada']=top_250['gross_us_canada'].str.replace('[,|$]', '' ,regex=True)
top_250['gross_us_canada'].fillna('0' , inplace=True)

In [21]:
def convert_runtime(runtime):
    match = re.match(r'(\d+)h?\s*(\d*)m?', runtime)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2)) if match.group(2) else 0
        return int(hours * 60 + minutes) 

In [22]:
top_250['runtime'] = top_250['runtime'].apply(convert_runtime)

In [23]:
all_persons = {}
for column in ['stars', 'writers', 'directors']:
    top_250[column] = top_250[column].apply(ast.literal_eval)
    for row in top_250[column]:
        for key, value in row.items():
            if key not in all_persons:
                all_persons[key] = value

In [24]:
unique_persons = pd.DataFrame(list(all_persons.items()), columns=['Person_ID', 'Name'])

In [25]:
unique_persons.head()

,Person_ID,Name
0,0000209,Tim Robbins
1,0000151,Morgan Freeman
2,0348409,Bob Gunton
3,0000008,Marlon Brando
4,0000199,Al Pacino


In [26]:
unique_persons.drop_duplicates(inplace=True)

In [68]:
top_250.head(10)

,title,runtime,parental_guide,year,genre,directors,stars,writers,gross_us_canada,film_id
0,The Shawshank Redemption,142,R,1994,['Drama'],{'0001104': 'Frank Darabont'},"{'0000209': 'Tim Robbins', '0000151': 'Morgan ...","{'0000175': 'Stephen King', '0001104': 'Frank ...",28767189,111161
1,The Godfather,175,R,1972,"['Crime', 'Drama']",{'0000338': 'Francis Ford Coppola'},"{'0000008': 'Marlon Brando', '0000199': 'Al Pa...","{'0701374': 'Mario Puzo', '0000338': 'Francis ...",136381073,68646
2,The Dark Knight,152,PG-13,2008,"['Action', 'Crime', 'Drama']",{'0634240': 'Christopher Nolan'},"{'0000288': 'Christian Bale', '0005132': 'Heat...","{'0634300': 'Jonathan Nolan', '0634240': 'Chri...",534987076,468569
3,The Godfather Part II,202,R,1974,"['Crime', 'Drama']",{'0000338': 'Francis Ford Coppola'},"{'0000199': 'Al Pacino', '0000134': 'Robert De...","{'0000338': 'Francis Ford Coppola', '0701374':...",47834595,71562
4,12 Angry Men,96,Approved,1957,"['Crime', 'Drama']",{'0001486': 'Sidney Lumet'},"{'0000020': 'Henry Fonda', '0002011': 'Lee J. ...",{'0741627': 'Reginald Rose'},0,50083
5,Schindler's List,195,R,1993,"['Biography', 'Drama', 'History']",{'0000229': 'Steven Spielberg'},"{'0000553': 'Liam Neeson', '0000146': 'Ralph F...","{'0447745': 'Thomas Keneally', '0001873': 'Ste...",96898818,108052
6,The Lord of the Rings: The Return of the King,201,PG-13,2003,"['Action', 'Adventure', 'Drama']",{'0001392': 'Peter Jackson'},"{'0000704': 'Elijah Wood', '0001557': 'Viggo M...","{'0866058': 'J.R.R. Tolkien', '0909638': 'Fran...",379427292,167260
7,Pulp Fiction,154,R,1994,"['Crime', 'Drama']",{'0000233': 'Quentin Tarantino'},"{'0000237': 'John Travolta', '0000235': 'Uma T...","{'0000233': 'Quentin Tarantino', '0000812': 'R...",107928762,110912
8,The Lord of the Rings: The Fellowship of the Ring,178,PG-13,2001,"['Action', 'Adventure', 'Drama']",{'0001392': 'Peter Jackson'},"{'0000704': 'Elijah Wood', '0005212': 'Ian McK...","{'0866058': 'J.R.R. Tolkien', '0909638': 'Fran...",316115420,120737
9,"The Good, the Bad and the Ugly",178,Approved,1966,"['Adventure', 'Western']",{'0001466': 'Sergio Leone'},"{'0000142': 'Clint Eastwood', '0908919': 'Eli ...","{'0898812': 'Luciano Vincenzoni', '0001466': '...",25100000,60196


In [14]:
top_250.to_csv('top_250_new.csv', index=False)

In [13]:
database = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Sh@0831@Sh'
)

In [14]:
mycursor = database.cursor()

In [24]:

mycursor.execute('create database IMDB_TOP_250')


DatabaseError: 1007 (HY000): Can't create database 'imdb_top_250'; database exists

In [15]:
mycursor.execute('USE IMDB_TOP_250')

In [ ]:
mycursor.execute('create table  movie (id VARCHAR(8) PRIMARY KEY , \
                 Title VARCHAR(128) NOT NULL,\
                 Year INT,\
                 Runtime INT,\
                 Parental_guide VARCHAR(8),\
                 Gross_us_canada INT)')


In [ ]:
mycursor.execute('create table person (id varchar(8) PRIMARY KEY ,\
                 Name varchar(32))')

In [ ]:
mycursor.execute('CREATE TABLE cast (id INT AUTO_INCREMENT PRIMARY KEY,\
    Movie_id VARCHAR(8),\
    Person_id VARCHAR(8),\
    FOREIGN KEY (Movie_id) REFERENCES movie(id),\
    FOREIGN KEY (Person_id) REFERENCES person(id)\
) AUTO_INCREMENT = 1')

In [ ]:
mycursor.execute('CREATE TABLE crew (id INT AUTO_INCREMENT PRIMARY KEY,\
                 Movie_id VARCHAR(8),\
                 Person_id VARCHAR(8),\
                 role VARCHAR(8),\
                 FOREIGN KEY (Movie_id) REFERENCES movie(id),\
                 FOREIGN KEY (Person_id) REFERENCES person(id)\
                 ) AUTO_INCREMENT = 1')

In [ ]:
mycursor.execute('CREATE TABLE genre_movie (id INT AUTO_INCREMENT PRIMARY KEY, \
                  Movie_id VARCHAR(8),\
                  Genre VARCHAR(16),\
                 FOREIGN KEY (Movie_id) REFERENCES movie(id)\
                 ) AUTO_INCREMENT = 1')

#### add data from dataframe into database

In [96]:
 top_250.isna().sum()

title              0
runtime            0
parental_guide     0
year               0
genre              0
directors          0
stars              0
writers            0
person_id          0
gross_us_canada    0
film_id            0
dtype: int64

In [27]:
movie_id = top_250['film_id'].to_list()
titles = top_250['title'].tolist()
years = top_250['year'].tolist()
runtimes = top_250['runtime'].tolist()
parental_guides = top_250['parental_guide'].tolist()
gross_us_canadas = top_250['gross_us_canada'].tolist()
person_id = unique_persons['Person_ID'].tolist()
person_name = unique_persons['Name'].tolist()

In [101]:
values = list(zip(movie_id,titles, years, runtimes, parental_guides, gross_us_canadas))

In [103]:
sql = "INSERT INTO movie (id,Title, Year, Runtime, Parental_guide, Gross_us_canada) VALUES (%s,%s, %s, %s, %s, %s)"
mycursor.executemany(sql, values)
database.commit()

In [28]:
values = list(zip(person_id,person_name))

In [29]:
sql = "INSERT INTO person (id,Name) VALUES (%s,%s)"
mycursor.executemany(sql, values)
database.commit()